In [1]:
# from map_gen import Maze
from datetime import datetime
from rig.type_casts import float_to_fp, fp_to_float, NumpyFloatToFixConverter, NumpyFixToFloatConverter
import os

In [2]:
c_dir = os.getcwd()
print(c_dir)

d:\PME_ITB\Backup\Tools\Reinforcement Learning\Maze Generator


In [3]:
def gen_path(c_dir, subdir_name):
    
    # Create path to sub directory
    subdir_path = os.path.join(c_dir, subdir_name)
    
    # Check if subdirectory exist. If doesn't exist generate new one.
    if os.path.isdir(subdir_path)==False:
        print("Folder '%s' does not exist. " % subdir_name, end = "")
        os.mkdir(subdir_path)
        print("Created a new folder named '%s'."% subdir_name)
    else:
        print("Folder named '%s' exist." % subdir_name)
        
    # Return geneated path    
    return subdir_path

def scan_file(dir):
    # Get list of maze config available in the folder
    list_files = os.listdir(dir)

    # Sort file
    list_files.sort()
    
    # # Filter to only bitstream files
    # mazeConfig_list = []
    # for file in maze_files:
    #     extension = os.path.splitext(file)[1]
    #     if (extension == ".txt"):
    #         mazeConfig_list.append(file)

    # Print all files
    print("%d maze confing file(s) detected."% len(list_files))
    for file in list_files:
        idx = list_files.index(file) + 1
        print("%d) %s"% (idx, file))
    
    return list_files

def select_file(dir):
    file_list = scan_file(dir)
    # Get user input to select overlay from overlay file list
    loop = True
    while(loop):
        idx = int(input('\nInput Select Index (1-%d) : '%(len(file_list))))-1
        if ((0 <= idx) and (idx < len(file_list))):
            loop = False
            selected_file = file_list[idx]
            print("\nSelected '%s'"% selected_file)
        else:
            print('Input out of range. Please try again.')
    return selected_file

def choose_config(c_dir, results_folder):
    results_dir = gen_path(c_dir, results_folder)
    target_file = select_file(results_dir)
    target_dir = gen_path(results_dir, target_file)
    
    list_of_file = scan_file(target_dir)

    # Filter to only bitstream files
    config_file_list = []
    for file in list_of_file:
        extension = os.path.splitext(file)[1]
        if (extension == ".txt"):
            config_file_list.append(file)
    
    if (len(config_file_list)!=1):
        print(f'Mutliple .txt files detected. Selected {config_file_list[0]}')
    else:
        print(f'Selected {config_file_list[0]}')
    config_file = config_file_list[0]

    return target_dir, config_file

def load_mazeConfig(maze_dir, config_file):
    # Read Maze config file
    config_target = os.path.join(maze_dir, config_file)
    with open(config_target, 'r') as f:
        print(f'Loading {config_file}...')
        lines = f.readlines()
        total_line = len(lines)
        print(f'\tFile consists of {total_line} lines of data.')
        f.close()
        
    # Load Maze Size
    maze_x = int(lines[0])
    maze_y = int(lines[1])
    total_state = maze_x * maze_y
    print(f'\tMaze size loaded. {maze_x}X{maze_y} ({total_state} states)')

    # Load total action
    total_act = int(lines[2])
    print(f'\tNumber of action loaded. There are {total_act} actions')

    # Load Next State list
    NS_list = [[0] * total_act for i in range(total_state)]
    for i in range(total_state):
        x = lines[3+i].split(';')
        x.remove('\n')
        for j in range(total_act):
            NS_list[i][j] = int(x[j])
    print('\tNext State list loaded.')

    # Load Current Reward List
    RT_list = [[0.0] * total_act for i in range(total_state)]
    for i in range(total_state):
        x = lines[3+total_state+i].split(';')
        x.remove('\n')
        for j in range(total_act):
            RT_list[i][j] = float(x[j])
    print('\tCurrent Reward list loaded.')
    print(f'Finish loading {config_file}')
    
    return total_state, total_act, NS_list, RT_list


In [4]:
config_dir, config_file = choose_config(c_dir, 'results')
config_data = load_mazeConfig(config_dir, config_file)

# Pass content of maze_config
total_state = config_data[0]
total_act = config_data[1]
ns_list = config_data[2]
rt_list = config_data[3]

Folder named 'results' exist.
10 maze confing file(s) detected.
1) 220830_10X10_0
2) 220830_10X10_1
3) 220830_10X10_2
4) 220830_10X10_3
5) 220830_10X10_4
6) 221013_05X05_0
7) 221013_05X05_1
8) 221013_05X05_2
9) 221013_05X05_3
10) 221013_05X05_4

Selected '221013_05X05_3'
Folder named '221013_05X05_3' exist.
3 maze confing file(s) detected.
1) 22101305X05_c3.svg
2) 22101305X05_s3.svg
3) 22101305X05c3.txt
Selected 22101305X05c3.txt
Loading 22101305X05c3.txt...
	File consists of 53 lines of data.
	Maze size loaded. 5X5 (25 states)
	Number of action loaded. There are 4 actions
	Next State list loaded.
	Current Reward list loaded.
Finish loading 22101305X05c3.txt


# Display NS Memory and RT Memory

In [5]:
print('NEXT STATE MEMORY')
for i in range(total_state):
    print(f'S{(i):03d}: ',end='')
    for j in range(total_act):
        print(f' {(ns_list[i][j]):03d} |',end='')
    print()

NEXT STATE MEMORY
S000:  005 | 001 | 000 | 000 |
S001:  001 | 002 | 000 | 001 |
S002:  007 | 003 | 001 | 002 |
S003:  008 | 003 | 002 | 003 |
S004:  009 | 004 | 004 | 004 |
S005:  010 | 005 | 005 | 000 |
S006:  011 | 006 | 006 | 006 |
S007:  007 | 007 | 007 | 002 |
S008:  013 | 008 | 008 | 003 |
S009:  014 | 009 | 009 | 004 |
S010:  015 | 010 | 010 | 005 |
S011:  011 | 012 | 011 | 006 |
S012:  012 | 013 | 011 | 012 |
S013:  013 | 013 | 012 | 008 |
S014:  019 | 014 | 014 | 009 |
S015:  015 | 016 | 015 | 010 |
S016:  021 | 016 | 015 | 016 |
S017:  022 | 018 | 017 | 017 |
S018:  018 | 019 | 017 | 018 |
S019:  024 | 019 | 018 | 014 |
S020:  020 | 021 | 020 | 020 |
S021:  021 | 022 | 020 | 016 |
S022:  022 | 022 | 021 | 017 |
S023:  023 | 024 | 023 | 023 |
S024:  024 | 024 | 023 | 019 |


In [6]:
print('REWARD MEMORY')
for i in range(total_state):
    print(f'S{(i):03d}: ',end='')
    for j in range(total_act):
        val = rt_list[i][j]
        if (val == -1):
            print(f'  {(val):.1f} |',end='')
        elif (val == -10):
            print(f' {(val):.1f} |',end='')
        else:
            print(f'  {(val):.1f} |',end='')
    print()

REWARD MEMORY
S000:   -1.0 |  -1.0 | -10.0 | -10.0 |
S001:  -10.0 |  -1.0 |  -1.0 | -10.0 |
S002:   -1.0 |  -1.0 |  -1.0 | -10.0 |
S003:   -1.0 | -10.0 |  -1.0 | -10.0 |
S004:   -1.0 | -10.0 | -10.0 | -10.0 |
S005:   -1.0 | -10.0 | -10.0 |  -1.0 |
S006:   -1.0 | -10.0 | -10.0 | -10.0 |
S007:  -10.0 | -10.0 | -10.0 |  -1.0 |
S008:   -1.0 | -10.0 | -10.0 |  -1.0 |
S009:   -1.0 | -10.0 | -10.0 |  -1.0 |
S010:   -1.0 | -10.0 | -10.0 |  -1.0 |
S011:  -10.0 |  -1.0 | -10.0 |  -1.0 |
S012:  -10.0 |  -1.0 |  -1.0 | -10.0 |
S013:  -10.0 | -10.0 |  -1.0 |  -1.0 |
S014:   -1.0 | -10.0 | -10.0 |  -1.0 |
S015:  -10.0 |  -1.0 | -10.0 |  -1.0 |
S016:   -1.0 | -10.0 |  -1.0 | -10.0 |
S017:   -1.0 |  -1.0 | -10.0 | -10.0 |
S018:  -10.0 |  -1.0 |  -1.0 | -10.0 |
S019:   -1.0 | -10.0 |  -1.0 |  -1.0 |
S020:  -10.0 |  -1.0 | -10.0 | -10.0 |
S021:  -10.0 |  -1.0 |  -1.0 |  -1.0 |
S022:  -10.0 | -10.0 |  -1.0 |  -1.0 |
S023:  -10.0 |  -1.0 | -10.0 | -10.0 |
S024:  -10.0 | -10.0 |  -1.0 |  -1.0 |


# Choose Goal

In [7]:
goal_state = int(input('Goal State: '))

print("Maze Goal is S{:03d}".format(goal_state))

# Updates Current Reward list
if (goal_state > (total_state-1)):
    print("Goal State cannot be used. Try again")
else:
    goal_reward = 10
    for i in range(total_state):
        for j in range(total_act):
            if (ns_list[i][j]==goal_state)&(i != goal_state):
                rt_list[i][j] = goal_reward
    print('Current Reward list updated.')

Maze Goal is S006
Current Reward list updated.


# Generate .coe Files

In [8]:
data_width = 16
fraction_bit = 10

In [9]:
coe_folder = f'coeS{total_state:03d}G{goal_state:03d}Q{data_width:02d}{fraction_bit:02d}'
# coe_dir = gen_path(config_dir, coe_folder)
target_dir = os.path.join(config_dir, coe_folder)

# Create the directory if it does not exist
if os.path.isdir(target_dir):
    print("File '% s' already exist" % target_dir)
else:
    os.mkdir(target_dir)
    print("File '% s' created" % target_dir)
## Generate 4 .coe files for NS_MEM
    for a in range(total_act):
        filename = f'S{total_state}_NS{a}_MEM.coe'
        path = os.path.join(target_dir, filename)
        with open(path, 'w') as f:
            print('memory_initialization_radix=10;', file = f)
            print('memory_initialization_vector=',  end ='', file = f)
            for s in range(total_state):
                print(f' {ns_list[s][a]}',  end ='', file = f)
            print(';', file = f)
        f.close()

## Generate a .coe file for RT_MEM
    filename = f'S{total_state}_RT_MEM.coe'
    path = os.path.join(target_dir, filename)
    with open(path, 'w') as f:
        print('memory_initialization_radix=10;', file = f)
        print('memory_initialization_vector=',  end ='', file = f)
        for s in range(total_state):
            for a in range(total_act):
                val0 = rt_list[s][a]*(2**fraction_bit)
                val1 = (int(val0) + (1 << data_width)) % (1 << data_width)
                print(f' {val1}',  end ='', file = f)
        print(';', file = f)
    f.close()

File 'd:\PME_ITB\Backup\Tools\Reinforcement Learning\Maze Generator\results\221013_05X05_3\coeS025G006Q1610' created


In [10]:
# target_folder = f'{maze_config}_G{goal_state:03d}_{data_width}_{fraction_bit}'
# target_dir = os.path.join(results_dir, target_folder)

# # Create the directory if it does not exist
# if os.path.isdir(target_dir):
#     print("File '% s' already exist" % target_dir)
# else:
#     os.mkdir(target_dir)
#     print("File '% s' created" % target_dir)

# ## Generate a .coe files for NS_MEM
#     filename = f'S{total_state}_NS.coe'
#     path = os.path.join(target_dir, filename)
#     with open(path, 'w') as f:
#         print('memory_initialization_radix=10;', file = f)
#         print('memory_initialization_vector=',  end ='', file = f)
#         for s in range(total_state):
#             for a in range(total_act):
#                 print(f' {NS_list[s][a]}',  end ='', file = f)
#         print(';', file = f)
#     f.close()

# ## Generate a .coe file for RT_MEM
#     filename = f'S{total_state}_RT_MEM.coe'
#     path = os.path.join(target_dir, filename)
#     with open(path, 'w') as f:
#         print('memory_initialization_radix=10;', file = f)
#         print('memory_initialization_vector=',  end ='', file = f)
#         for s in range(total_state):
#             for a in range(total_act):
#                 val0 = RT_list[s][a]*(2**fraction_bit)
#                 val1 = (int(val0) + (1 << data_width)) % (1 << data_width)
#                 print(f' {val1}',  end ='', file = f)
#         print(';', file = f)
#     f.close()